# Homework 7

## CSCI E-82A


In the a previous homework assignment, you used two different dynamic programming algorithms to solve a robot navigation problem by finding optimal paths to a goal in a simplified warehouse environment. Now you will use first visit Monte Carlo reinforcement learning to find optimal paths in the same environment.

The configuration of the warehouse environment is illustrated in the figure below.

<img src="GridWorldFactory.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid World for Factory Navigation Example** </center>

The goal is for the robot to deliver some material to position (state) 12, shown in blue. Since there is a goal state or **terminal state** this an **episodic task**. 

There are some barriers comprised of the states $\{ 6, 7, 8 \}$ and $\{ 16, 17, 18 \}$, shown with hash marks. In a real warehouse, these positions might be occupied by shelving or equipment. We do not want the robot to hit these barriers. Thus, we say that transitioning to these barrier states is **taboo**.

As before, we do not want the robot to hit the edges of the grid world, which represent the outer walls of the warehouse. 

## Representation

As with many such problems, the starting place is creating the **representation**. In the cell below encode your representation for the possible action-state transitions. From each state there are 4 possible actions:
- up, u
- down, d,
- left, l
- right, r

There are a few special cases you need to consider:
- Any action transitioning state off the grid or into a barrier should keep the state unchanged. 
- Any action in the goal state keeps the state unchanged. 
- Any transition within the taboo (barrier) states can keep the state unchanged. If you experiment, you will see that other encodings work as well since the value of a barrier states are always zero and there are no actions transitioning into these states. 

> **Hint:** It may help you create a pencil and paper sketch of the transitions, rewards, and probabilities or policy. This can help you to keep the bookkeeping correct. 

In [1]:
## import numpy for latter
import numpy as np
import numpy.random as nr



You need to define the initial transition probabilities for the Markov process. Set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. 

> **Note:** As these are just starting values, the exact values of the transition probabilities are not actually all that important in terms of solving the MC RL problem. Also, notice that it does not matter how the taboo state transitions are encoded. The point of the MC RL algorithm is to learn a model including the transition policy.

The robot receives the following rewards:
- 10 for entering position 12. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the MC RL agent**. The agent must **learn** the rewards by sampling the environment. 

In the code cell below encode your representation of this reward structure you will use in your simulated environment.  

You will find it useful to create a list of taboo states, which you can encode in the cell below.

## Policy Evaluation

With your representations defined, you can now create and test functions to perform MC RL **policy evaluation**. You will need these functions for your policy improvement code. 

As a first step you will need a function to generate episodes given the policy in the warehouse environment. You are welcome to start with the `MC_generate_episode` function from the MC RL notebook. However, keep in mind that you must modify this code to correctly treat the taboo states of the barrier. Specifically, taboo states will not be visited on the random walk. 

Set a seed of `4567` and execute your code to test it.  

Examine your results and answer the following questions to ensure your code has operated correctly:
1. Have any taboo states been visited? ANS:
2. Does the random walk end at the terminal state? ANS: 

Next, you need to create a function to compute the first visit action values along the random walk path. You are welcome to start with the `MC_state_values` function from the MC RL notebook. Notice that this function is incorrectly named, as it actually does compute action values.  

Execute your function for 1,000 episodes. Make sure your code has 0 action values for the taboo states and the goal (terminal) state. If not, something is likely wrong with your random walk generation. 

Examine your results and answer the following questions to ensure you action value function operates correctly:
1. Are the values of the taboo states 0? ANS: 
2. Are the states with the highest values adjacent to the terminal state? ANS: 
3. Are the values of the states decreasing as the distance from the terminal state increases? ANS: 


## Policy Improvement

Now that you have your representation and a functions to perform the MC policy evaluation you have everything you need to apply the policy improvement algorithm to create an optimal policy for the robot to reach the goal. 

If your policy evaluation functions work correctly, you should be able to use the `MC_optimal_policy` function from the MC RL notebook. Execute your code using 5 cycles of 1000 episodes for policy improvement. Set $\epsilon = 0.01$. Computation may take some time. Make sure you print the state values as well as the policy you discovered. 

> **Note:** There is some chance the algorithm will not converge from certain starting states. This seems to result from accumulating small errors over the many iterations. The solution should be to change seeds. 

NameError: name 'i' is not defined

Examine your results and answer the following questions:
1. Based on the action values, does it appear that the algorithm is near convergence with only small changes? ANS: 
2. Assume the robot enters the warehouse though a door at state (position) 2. As you did for the DP exercise, create an in illustration of the optimal path to the goal based on the policy your algorithm discovered. Replace the figure below with your illustration.

<img src="GridWorldFactory.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Replace this diagram with one showing your solution** </center>

Finally, test your improved policy by computing the action values. In the cell below, create and execute the code to display the action values for all the states. 

Examine your results and answer the following questions. 

1. Do your action values represent a significant improvement over the initial random policy? ANS: 
2. Do the states near the goal have action values near the terminal reward? ANS: 